### Consolidate the holding and analytics accorss IBKR and RobinHood

In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
import pandas_datareader
import datetime as dt
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

import robin_stocks.robinhood as robin
from IPython.display import display, HTML
import plotly.express as px

pd.options.display.max_rows = 400
pd.options.display.max_columns = 400
pd.options.display.float_format = '{:,.2f}'.format
display(HTML("<style>.container { width:96% !important; }</style>"))
display(HTML("<style>div.output_scroll { height: 50em; }</style>"))

In [ ]:
import import_ipynb
from general_functions import *

In [ ]:
robin.login('username','password')

In [ ]:
def portfolio_history():
    """Get portfolio daily value and return 
    adjusted_close_equity: include total portfolio value (NAV) including Cash
    """
    df_portfolio_hist = pd.DataFrame(robin.get_historical_portfolio(interval='day',span='5year')['equity_historicals'])
    list_of_col_to_be_float= ['adjusted_open_equity', 'adjusted_close_equity', 'open_equity',
     'close_equity', 'open_market_value', 'close_market_value','net_return']
    ### data processing
    df_portfolio_hist[list_of_col_to_be_float]=df_portfolio_hist[list_of_col_to_be_float].astype(float)
    df_portfolio_hist['begins_at'] = pd.to_datetime(df_portfolio_hist['begins_at'])
    df_portfolio_hist['begins_at'] = pd.to_datetime(df_portfolio_hist['begins_at'].dt.date)
    ### metric calculation
    df_portfolio_hist['PnL'] =  df_portfolio_hist['adjusted_close_equity'].diff()
    df_portfolio_hist['CumPnL'] =  df_portfolio_hist['PnL'].fillna(0).cumsum()
    df_portfolio_hist['Return'] = df_portfolio_hist['adjusted_close_equity'].pct_change().fillna(0)
    df_portfolio_hist['CumReturn']=(((df_portfolio_hist['adjusted_close_equity'].pct_change()).fillna(0)+1).cumprod()-1)*100
    return(df_portfolio_hist)

In [ ]:
def trade_histroy():
    """Get all trades ever made | buy, sell"""
    df_trade_data = pd.DataFrame(robin.get_all_stock_orders())
    df_trade_data = symbol_and_name_adder(df_trade_data)
    df_trade_data[['cumulative_quantity','average_price']] = df_trade_data[['cumulative_quantity','average_price']].astype(float)
    df_trade_data['last_transaction_at'] = df_trade_data['last_transaction_at'].apply(lambda x: pd.to_datetime(x))
    df_trade_data['updated_at'] = df_trade_data['updated_at'].apply(lambda x: pd.to_datetime(x))
    df_trade_data['date'] = pd.to_datetime(df_trade_data['updated_at'].dt.to_period("d").apply(lambda x: str(x)[:10]))
    df_trade_data['trade_amount'] = df_trade_data['cumulative_quantity']* df_trade_data['average_price']
    df_trade_summary = data_frame_flattener(pd.pivot_table(df_trade_data,
                   index='date',
                   values=['trade_amount'],
                   columns=['side'],
                   aggfunc='sum').fillna(0).reset_index())
    df_trade_summary['Net'] = df_trade_summary['trade_amount buy'] - df_trade_summary['trade_amount sell']
    df_trade_summary['trade_amount sell'] = df_trade_summary['trade_amount sell'] * -1
    return({"df_trade_data":df_trade_data,
            "df_trade_summary":df_trade_summary})

In [ ]:
def get_name_and_symbol(url):
    symbol = robin.get_symbol_by_url(url)
    name = robin.get_name_by_url(url)
    return({"instrument":url,"symbol":symbol,"name":name})


def symbol_and_name_finder(df_data):
    list_of_symbol_and_name =[]
    list_of_instrument =list(set(df_data['instrument']))
    for instrument in list_of_instrument:
        res = get_name_and_symbol(instrument)
        #print(res)
        list_of_symbol_and_name.append(res)
    df_name_symbol=pd.DataFrame(list_of_symbol_and_name)
    return(df_name_symbol)


def symbol_and_name_adder(df):
    df_data = df.copy()
    df_name_symbol = symbol_and_name_finder(df_data)
    df_data = pd.merge(df_data,
             df_name_symbol,
            left_on = "instrument",
            right_on = "instrument",
            how='left')
    return(df_data)

In [ ]:
def fundamental_from_robinhood(list_of_symbol):
    list_of_fundamental_from_robin = []
    for symbol in list_of_symbol:
        list_of_fundamental_from_robin.append(robin.get_fundamentals(symbol)[0])
    df_fundemental_from_robin = pd.DataFrame(list_of_fundamental_from_robin)[['symbol','sector','industry','market_date',
                                                                              'market_cap','shares_outstanding',
                                                                              'dividend_yield','pb_ratio','year_founded',
                                                                              'num_employees','ceo']]
    return(df_fundemental_from_robin)

In [ ]:
def my_current_holdings():
    my_stocks = robin.build_holdings()
    df_my_portfolio = pd.DataFrame(my_stocks).T.reset_index().rename(columns={"index":"symbol"})
    ### pull fundeentlam from Robinhood
    list_of_symbol_in_my_holding = list(set(df_my_portfolio['symbol']))
    df_fundemental_from_robin = fundamental_from_robinhood(list_of_symbol_in_my_holding)
    df_my_portfolio = pd.merge(df_my_portfolio,
                               df_fundemental_from_robin,
                               left_on="symbol",
                               right_on='symbol',
                               how='left')
    ### List of col to change to float
    list_of_quant_col = ['price', 'quantity', 'average_buy_price', 'equity',
                         'percent_change', 'intraday_percent_change', 'equity_change','pe_ratio', 'percentage',
                        'market_cap','shares_outstanding','dividend_yield','pb_ratio','year_founded','num_employees']
    df_my_portfolio[list_of_quant_col] = df_my_portfolio[list_of_quant_col].astype(float)
    df_my_portfolio = df_my_portfolio.sort_values(['equity'],ascending=False).reset_index(drop=True)
    ### percentage calucaltion is not correct
    df_my_portfolio['equity_initial'] = df_my_portfolio['equity'] - df_my_portfolio['equity_change']
    df_my_portfolio['percentage'] = (df_my_portfolio['equity']/df_my_portfolio['equity'].sum())*100
    df_my_portfolio['CumPercent'] = df_my_portfolio['percentage'].cumsum()
    ### pull fundeentlam from Yahoo
    print("Getting holding attributes form yahoo finance")
    df_symbol_attributes_from_yahoo = symbol_attribute_retriever_from_yahoo(list_of_symbol_in_my_holding)
    
    df_my_portfolio = pd.merge(df_my_portfolio,
                               df_symbol_attributes_from_yahoo,
                               left_on=['symbol'],
                               right_on=['symbol'],
                               how='left',
                               suffixes=("_robin","_yahoo"))

    df_my_portfolio['name_robin'] =df_my_portfolio['name'].copy()
    df_my_portfolio['industry'] = df_my_portfolio['industry_yahoo'].fillna(df_my_portfolio['industry_robin'])
    df_my_portfolio['sector'] = df_my_portfolio['sector_yahoo'].fillna(df_my_portfolio['sector_robin'])
    df_my_portfolio['name'] = df_my_portfolio['shortName'].fillna(df_my_portfolio['name_robin'])
    df_my_portfolio['pb_ratio'] = df_my_portfolio['pb_ratio'].fillna(df_my_portfolio['pb_ratio'])
    df_my_portfolio['trailingPE'] = df_my_portfolio['trailingPE'].fillna(df_my_portfolio['pe_ratio'])
    df_my_portfolio['dividendYield'] = df_my_portfolio['dividendYield'].fillna(df_my_portfolio['dividend_yield'])
    df_my_portfolio['fullTimeEmployees'] = df_my_portfolio['fullTimeEmployees'].fillna(df_my_portfolio['num_employees'])
    print("retrived attributes form yahoo finance")
    

    ### orgonize columns
    cols_to_return=['symbol', 'name', 'sector', 'industry', 'returnOnAssets','returnOnEquity','quantity', 
                    'equity','equity_initial','equity_change','percent_change','percentage','CumPercent',
                    'average_buy_price','price','ev_ebit','firm_to_ebit','trailingPE','forwardPE','bookValue','priceToBook',
                    'ev','marketCap','totalDebt','totalCash','totalRevenue','ebit','netDebt','grossMargins','operatingMargins','profitMargins',
                    'currentRatio','quickRatio',
                    'grossProfits','ebitda','operatingCashflow','ebitdaMargins','freeCashflow',
                    'heldPercentInsiders','dividendYield','fiveYearAvgDividendYield','payoutRatio','symbolCurrency',
                    #'longBusinessSummary',
                    'fullTimeEmployees','debtToEquity','financialCurrency','exchange']
    df_my_portfolio = df_my_portfolio[cols_to_return].copy()
    ### rename returned columsn
    df_my_portfolio.rename(columns={"returnOnAssets":"ROA",
                                            "returnOnEquity":"ROE",
                                            "average_buy_price":"ave_price",
                                             "bookValue":"pb",
                                             "priceToBook":"pb_ratio",
                                             "trailingPE":"pe",
                                             "forwardPE":'fpe',
                                            'totalRevenue':'rev',
                                            'operatingCashflow':'ocf',
                                            'dividendYield':'div_yeild',
                                         'fiveYearAvgDividendYield':'5y_div_yeild'},inplace=True)
    return(df_my_portfolio)

In [ ]:
def my_open_orders(add_current_holding=True):
    open_orders = robin.get_all_open_stock_orders()
    df_open_order_stock = pd.DataFrame(open_orders)
    df_open_posotion = symbol_and_name_adder(df_open_order_stock)
    df_open_posotion[['price','quantity']] = df_open_posotion[['price','quantity']].astype(float)
    df_open_posotion['amount'] = df_open_posotion['price'] * df_open_posotion['quantity']
    ### get current price of the securyties
    list_of_symbols_with_open_orders = list(set(df_open_posotion['symbol']))
    list_of_current_price = []
    list_of_symbols_with_open_orders_with_current_price = []
    print("getting current prices")
    for symbol in list_of_symbols_with_open_orders:
        current_price = robin.get_latest_price([symbol])[0]
        list_of_current_price.append(current_price)
        list_of_symbols_with_open_orders_with_current_price.append(symbol)
    df_current_prices  = pd.DataFrame({"symbol":list_of_symbols_with_open_orders_with_current_price,
                                       "current_price":list_of_current_price})
    df_open_posotion = pd.merge(df_open_posotion,df_current_prices,left_on="symbol",right_on='symbol',how='left')
    if add_current_holding:
        print("Adding Current Holdings")
        df_my_current_holding = my_current_holdings()
        df_my_current_holding_to_be_merged_with_open_orderd=column_suffix_adder(df_my_current_holding[['symbol','quantity','ave_price','equity']],
                                                                          list_of_columns_to_add_suffix_on=['quantity','ave_price','equity'],
                                                                          suffix="_current_holding")
        df_open_posotion = pd.merge(df_open_posotion,
                 df_my_current_holding_to_be_merged_with_open_orderd,
                left_on="symbol",right_on='symbol',how='left')
        df_open_posotion = df_open_posotion[
            ['symbol','name','side','type','current_price','price','quantity','amount',
             'quantity_current_holding','ave_price_current_holding','equity_current_holding',
             'state','created_at']].copy()
        return(df_open_posotion)
    df_open_posotion = df_open_posotion[
            ['symbol','name','side','type','current_price','price','quantity','amount',
             'state','created_at']].copy()
    return(df_open_posotion)

In [ ]:
def my_portfolio():
    ### Get current holdings
    df_my_current_holding = my_current_holdings()
    ### Get Open orders
    df_open_posotion = my_open_orders(add_current_holding=False)
    ## Megre current holdings with open position
    df_my_current_holding_to_be_merged_with_open_orderd= column_suffix_adder(df_my_current_holding[['symbol','quantity','ave_price','equity']],
                                                                          list_of_columns_to_add_suffix_on=['quantity','ave_price','equity'],
                                                                          suffix="_current_holding")
    df_open_posotion = pd.merge(df_open_posotion,
                                df_my_current_holding_to_be_merged_with_open_orderd,
                                left_on="symbol",right_on='symbol',how='left')
    df_open_posotion = df_open_posotion[['symbol','name','side','type','current_price','price','quantity','amount',
                                         'quantity_current_holding','ave_price_current_holding','equity_current_holding',
                                         'state','created_at']].copy()
    ### Get Portfolio History
    df_portfolio_hist = portfolio_history()
    print("Portfolio Data Retrival Compeleted")
    return({"my_current_holdings":df_my_current_holding,
            "open_orders":df_open_posotion,
            "portfolio_history":df_portfolio_hist})

###  How to use it

In [ ]:
my_portfolio_dict  = my_portfolio()
trade_histroy_dict= trade_histroy()

In [ ]:
portfolio_analytics(my_portfolio_dict,trade_histroy_dict)